In [7]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import csv
import pandas as pd
import pickle

import spacy

max_length = 100

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
with open('vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
        
len(vocab)
# vocab.get('->', -1), vocab

69256

In [3]:
vocab.get('<---"<user>', -1)

65903

# Load Embeddings

In [4]:
print("loading embeddings")
#     with open('cooc.pkl', 'rb') as f:
#         cooc = pickle.load(f)
embeddings = np.load('embeddings.npy')

loading embeddings


In [5]:
embeddings.shape

(69256, 20)

# Loading training set

In [33]:
#Read Data neg and pos with corresponding label

raw_data_neg = pd.read_csv('train_neg.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=True, warn_bad_lines=True, quoting=csv.QUOTE_NONE).drop_duplicates()
# raw_data_neg['text'] = raw_data_neg['text']
raw_data_neg['label'] = 0
# raw_data_neg = raw_data_neg[:5000]

raw_data_pos = pd.read_csv('train_pos.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=True, warn_bad_lines=True, quoting=csv.QUOTE_NONE).drop_duplicates()
raw_data_pos['label'] = 1
# raw_data_pos = raw_data_pos[:5000]


raw_data_train = pd.concat([raw_data_neg, raw_data_pos], ignore_index=True)


# raw_data_test = pd.read_csv('test_data.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
#                                error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()


# raw_data_test['id'], raw_data_test['text'] = raw_data_test['text'].apply(lambda x: x.split(',')[0]), raw_data_test['text'].apply(lambda x: ','.join(x.split(',')[1:]))

In [34]:
indices = np.arange(raw_data_train.shape[0])
np.random.shuffle(indices), raw_data_train.shape[0]

raw_data_train.loc[indices].text, raw_data_train.loc[indices].label

raw_data_train.text = raw_data_train.loc[indices].text.values
raw_data_train.label = raw_data_train.loc[indices].label.values

In [35]:
raw_data_train

,text,label
0,<user> you're so beautiful-wish i was . x,0
1,<user> oh my god yea please ! ! we'll all be 2...,1
2,i want to go the capital summer time ball,0
3,omen rt <user> the devil inside rt <user> what...,0
4,<user> i miss you too fool ! ! i'm sure i'll r...,1
5,cute japanese credit card cover sticker ( pape...,0
6,<user> it's no use having half an actimel pot ...,1
7,video : hot mess of the week : worst twerk tea...,1
8,<user> omg ! thats fantastic ! ! how did it go...,1
9,"<user> and now you want a gimp mask , all link...",1


In [4]:
#Read Data neg and pos with corresponding label

raw_data_neg = pd.read_csv('train_neg_full.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
# raw_data_neg['text'] = raw_data_neg['text']
raw_data_neg['label'] = 0
# raw_data_neg = raw_data_neg[:5000]

raw_data_pos = pd.read_csv('train_pos_full.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()
raw_data_pos['label'] = 1
# raw_data_pos = raw_data_pos[:5000]


raw_data_train = pd.concat([raw_data_neg, raw_data_pos], ignore_index=True)


raw_data_test = pd.read_csv('test_data.txt', header=None, sep="\n", encoding='utf-8', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE).drop_duplicates()


raw_data_test['id'], raw_data_test['text'] = raw_data_test['text'].apply(lambda x: x.split(',')[0]), raw_data_test['text'].apply(lambda x: ','.join(x.split(',')[1:]))

In [2]:
docs = list(nlp.pipe(raw_data_train['text'], batch_size=1000, n_threads=8))


features_train = np.zeros((len(docs), 300))

for (i, doc) in enumerate(docs):
    features_train[i, :] = doc.vector
    
features_train

# test_docs = list(nlp.pipe(raw_data_test['text'], batch_size=1000, n_threads=8))

NameError: name 'nlp' is not defined

In [12]:
test_docs = list(nlp.pipe(raw_data_test['text'], batch_size=1000, n_threads=8))

In [14]:
len(test_docs)

with open('test_docs_saved', 'wb') as f:
    pickle.dump(test_docs, f, pickle.HIGHEST_PROTOCOL)

In [15]:
with open('test_docs_saved', 'rb') as f:
        test_docs_loaded = pickle.load(f)


In [22]:
with open('train_docs_saved', 'wb') as f:
    pickle.dump(docs, f, pickle.HIGHEST_PROTOCOL)

In [21]:
len(test_docs_loaded), len(test_docs)

(10000, 10000)

In [23]:
# nlp(raw_data_train.text[0]).vector
# nlp(raw_data_train.text)

# docs = list(nlp.pipe(raw_data_train['text'], batch_size=1000, n_threads=8))

# test_docs = list(nlp.pipe(raw_data_test['text'], batch_size=1000, n_threads=8))
# print(max([len(x) for x in docs]))

# np.zeros((raw_data_test['processed_text'].shape[0], max_length))

features_train = np.zeros((len(docs), 300))

for (i, doc) in enumerate(docs):
    features_train[i, :] = doc.vector
    
features_train



features_test = np.zeros((len(test_docs), 300))

for (i, doc) in enumerate(test_docs):
    features_test[i, :] = doc.vector
    
    
features_train.shape, features_test.shape

((2270482, 300), (10000, 300))

In [24]:
X_train, y_train = features_train, raw_data_train.label
X_test = features_test

In [35]:
X_train.shape, y_train.shape, X_test.shape

((2270482, 300), (2270482,), (10000, 300))

# Construct Features for the Training Texts
convert each word in each tweet to a number (which indicates how frequent it is)

In [59]:
# raw_data.text[0].strip().split()
# raw_data_train['processed_text'] = raw_data_train.text.apply(lambda x: x.strip().split()).apply(lambda x: np.array([vocab.get(y, -1) for y in x if vocab.get(y, -1) >= 0]))
# raw_data_test['processed_text'] = raw_data_test.text.apply(lambda x: x.strip().split()).apply(lambda x: np.array([vocab.get(y, -1) for y in x if vocab.get(y, -1) >= 0]))

raw_data_train['processed_text'] = raw_data_train.text.apply(lambda x: x.strip()).apply(lambda x: nlp(x))
raw_data_test['processed_text'] = raw_data_test.text.apply(lambda x: x.strip()).apply(lambda x: nlp(x))


raw_data_train.processed_text
raw_data_test.processed_text

KeyboardInterrupt: 

Average word vectors over all words of the tweet

In [9]:
raw_data_train.processed_text[0], raw_data_test.processed_text[0]

(array([  552,    76, 16312,   466,    17,   466, 12655,    62,     9,
            7, 20052,  3368,    33,   170,  8252,    23,    25]),
 array([ 2653,  2824,  2265,  2653,  8695,    76,  2070,    26,     7,
        17085,  1130,     6,   378,  1318,    16,     7,   786,     9,
           23,    25]))

In [10]:
# raw_data_test[raw_data_test['id']== 1]
raw_data_test.loc[4588]

text              <user> its freezing haha bas helw helw .. ta3a...
id                                                             4589
processed_text    [0, 85, 5862, 64, 8497, 39, 4, 60, 37, 3, 93, ...
Name: 4588, dtype: object

In [72]:
features_train.shape, raw_data_train.label.shape

((2270482, 300), (2270482,))

In [33]:
features_train = np.zeros((raw_data_train['processed_text'].shape[0], max_length))
average_word_vectors_train = raw_data_train['processed_text'].apply(lambda x: embeddings[x, :] if len(x) > 0 else embeddings.mean(0).reshape(1, -1)).apply(lambda x: x.mean(0))

for (i, doc) in enumerate(average_word_vectors_train):
    features_train[i, :doc.shape[0]] = doc


X_train, y_train = features_train, raw_data_train.label


features_test = np.zeros((raw_data_test['processed_text'].shape[0], max_length))
# average_word_vectors_test = raw_data_test['processed_text'].apply(lambda x: embeddings[x, :] if len(x) > 0 else np.zeros(max_length)).apply(lambda x: x.mean(0))
average_word_vectors_test = raw_data_test['processed_text'].apply(lambda x: embeddings[x, :] if len(x) > 0 else embeddings.mean(0).reshape(1, -1)).apply(lambda x: x.mean(0))

# for (i, asd) in enumerate(raw_data_test['processed_text']):
#     print("i: ", i)
#     print(embeddings[asd, :], asd.shape)



for (i, doc) in enumerate(average_word_vectors_test):
#     print("i: ", i, doc)
    features_test[i, :doc.shape[0]] = doc

average_word_vectors_test

X_test = features_test

In [34]:
X_train.shape, y_train.shape, embeddings.shape, X_test.shape

NameError: name 'embeddings' is not defined

# Train a Linear Classifier
Let's start with logistic regression

In [53]:
from sklearn import svm

# model = LogisticRegression(random_state=0, solver='sag', multi_class='ovr', max_iter=1000, )
# model = svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear')
model = svm.LinearSVC(dual=False, max_iter=10000)


fitted_model = model.fit(X_train, y_train)

# Prediction
Predict labels for all tweets in the test set.

In [107]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
#             print(y_pred)
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})


In [55]:
y_test = fitted_model.predict(X_test)

In [56]:
y_train_predict = fitted_model.predict(X_train)

In [57]:
np.sum((y_train == y_train_predict)) / y_train.shape[0]

0.7747337349514333

In [43]:
y_test[y_test==0] = -1

In [44]:
y_test

array([-1,  1, -1, ..., -1,  1, -1])

In [45]:
create_csv_submission(raw_data_test['id'], y_test, 'logistic_regression_submission_2_full_dataset.csv')

In [67]:
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import LabelEncoder

import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Flatten, Conv1D, MaxPooling1D, GlobalAveragePooling1D, LSTM, merge
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.constraints import maxnorm
from keras.regularizers import l2
from keras import backend as K

len(docs), y_train.shape

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(2270482, (1816385,))

In [63]:
def extract_features(docs, max_length):
    docs = list(docs)
    X = np.zeros((len(docs), max_length), dtype='int32')
    for i, doc in enumerate(docs):
        j = 0
        for token in doc:
            if token.has_vector and not token.is_punct and not token.is_space:
                X[i, j] = token.rank + 1
                j += 1
                if j >= max_length:
                    break
    return X

In [81]:
# y = raw_data['label'].values
# y = y_train
y = raw_data_train.label

# Pull the raw_data into vectors
X = extract_features(docs, max_length=max_length)

# Split into train and test sets
rs = ShuffleSplit(n_splits=2, random_state=42, test_size=0.2)
train_indices, test_indices = next(rs.split(X))

X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]

# docs = np.array(docs, dtype=object)
# docs_train = docs[train_indices]
# docs_test = docs[test_indices]

# numeric_data = X_train, y_train, X_test, y_test
# raw_data = docs_train, docs_test

In [82]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1816385, 100), (1816385,), (454097, 100), (454097,))

In [83]:
def get_embeddings(vocab):
    max_rank = max(lex.rank+1 for lex in vocab if lex.has_vector)
    vectors = np.ndarray((max_rank+1, vocab.vectors_length), dtype='float32')
    for lex in vocab:
        if lex.has_vector:
            vectors[lex.rank + 1] = lex.vector
    return vectors


vocab_nlp = spacy.load('en_core_web_lg', parser=False, tagger=False, entity=False)
print('Preparing embeddings...')
embeddings = get_embeddings(vocab_nlp.vocab)

Preparing embeddings...


In [84]:
def build_model_twitter5(max_length=100,
                nb_filters=64,
                kernel_size=3,
                pool_size=2,
                regularization=0.01,
                weight_constraint=2.,
                dropout_prob=0.4,
                clear_session=True):
    if clear_session:
        K.clear_session()

    model = Sequential()
    model.add(Embedding(
        embeddings.shape[0],
        embeddings.shape[1],
        input_length=max_length,
        trainable=False,
        weights=[embeddings]))

    model.add(Conv1D(padding = "same", kernel_size = 3, filters = 32, activation = "relu"))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(LSTM(100))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model

In [169]:
embeddings.shape

(684825, 300)

In [86]:
#Model 5
np.random.seed(42)


epochs = 16
batch_size = 160


# Take a look at the shapes
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

tb_callback = keras.callbacks.TensorBoard(
        histogram_freq=0, write_graph=True)

model = build_model_twitter5()
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          batch_size=batch_size, epochs=epochs,
          callbacks=[tb_callback])

X_train: (1816385, 100)
y_train: (1816385,)
X_test: (454097, 100)
y_test: (454097,)
Train on 1816385 samples, validate on 454097 samples
Epoch 1/16
1816385/1816385 [==============================] - 2071s - loss: 0.6933 - acc: 0.5004 - val_loss: 0.6935 - val_acc: 0.4970
Epoch 2/16
  93600/1816385 [>.............................] - ETA: 1535s - loss: 0.6933 - acc: 0.5005

KeyboardInterrupt: 

In [157]:
from feature_extraction import train_test_features, dumpFeatures
import _pickle as cPickle

# train_sequences, test_sequences, labels, max_features = train_test_features(full=False, n_gram=True, pretrained=False, nb_words=None)

dumpFeatures(full=False, n_gram=True, pretrained=False, nb_words=20000, namefile='bigram_features.dat')

[X_train, y, X_test, max_features] = cPickle.load(open("bigram_features.dat", "rb"))

Processing of200000tweets for the training set...
200000/200000 processed tweets.
Processing of10000tweets for the testing set...
10000/10000 processed tweets.
Tokenization of tweets...


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/text.py:89: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 96338 unique tokens.
Adding 2-gram features
Average train sequence length: 29
Average test sequence length: 27
Pad sequences (samples x time)
train_sequences shape: (200000, 60)
test_sequences shape: (10000, 60)
Shuffling of the training set...


In [158]:
# train_sequences.shape, test_sequences.shape, labels.shape

X_train.shape, y.shape, X_test.shape, max_features

((200000, 60), (200000,), (10000, 60), 663394)

In [127]:
model_alt = svm.LinearSVC(dual=False, max_iter=10000)

# fitted_model_alt = model_alt.fit(train_sequences, test_sequences)
fitted_model_alt = model_alt.fit(X_train, y)

In [128]:
# y_test = fitted_model_alt.predict(labels)
y_test = fitted_model_alt.predict(X_test)

y_train_predict = fitted_model_alt.predict(X_train)

# np.sum((test_sequences == y_train_predict)) / y_train.shape[0]
np.sum((y == y_train_predict)) / y.shape[0]

0.59497

In [ ]:
y_test[y_test==0] = -1

In [129]:
model = Sequential()
model.add(Embedding(max_features+1, 50, input_length=X_train.shape[1]))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 60, 50)            33169750  
_________________________________________________________________
global_average_pooling1d_1 ( (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 33,169,801
Trainable params: 33,169,801
Non-trainable params: 0
_________________________________________________________________
None


In [130]:
model.fit(X_train, y, validation_split=0.1, nb_epoch=2, batch_size=128, verbose=1)

train_6 = model.predict_proba(X_train, batch_size=128)
test_6 = model.predict_proba(X_test)

/usr/local/lib/python3.6/dist-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 180000 samples, validate on 20000 samples
Epoch 1/2
180000/180000 [==============================] - 376s - loss: 0.4818 - acc: 0.7731 - val_loss: 0.3874 - val_acc: 0.8248
Epoch 2/2
 9312/10000 [==========================>...] - ETA: 0s

In [142]:
test_6_labels = model.predict(X_test)

In [144]:
test_6.shape

# cPickle.dump(test_6, open('test_fasttext_bigram.dat', 'wb'))
test_6_labels[test_6_labels >= 0.5] = 1
test_6_labels[test_6_labels < 0.5] = -1

In [150]:
test_6_labels_reversed = test_6_labels.copy()

test_6_labels_reversed[test_6_labels_reversed == -1] = 0
test_6_labels_reversed[test_6_labels_reversed == 1] = -1
test_6_labels_reversed[test_6_labels_reversed == 0] = 1

test_6_labels_reversed

array([[-1.],
       [-1.],
       [-1.],
       ...,
       [-1.],
       [ 1.],
       [-1.]], dtype=float32)

In [151]:
test_6_labels[test_6_labels == -1].shape

create_csv_submission(raw_data_test['id'], test_6_labels_reversed, 'NN_submission_1_partial_dataset.csv')

In [141]:
import xgboost as xgb

# train = np.hstack((train_6, train_6))
# test = np.hstack((test_6, test_6))

y = np.array(1250000 * [0] + 1250000 * [1])
np.random.seed(0)
np.random.shuffle(y)

"""
Fitting XGBOOST over the probability matrix obtained from our models
"""

model = xgb.XGBClassifier().fit(train_6, y)

y_pred = model.predict(test)
y_pred = 1 - 2*y_pred

XGBoostError: b'[11:27:17] /workspace/src/objective/regression_obj.cu:66: Check failed: preds.Size() == info.labels_.Size() (200000 vs. 2500000) labels are not correctly providedpreds.size=200000, label.size=2500000\n\nStack trace returned 10 entries:\n[bt] (0) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(dmlc::StackTrace()+0x3d) [0x7f136d1e95cd]\n[bt] (1) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x18) [0x7f136d1e99c8]\n[bt] (2) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::obj::RegLossObj<xgboost::obj::LogisticClassification>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*)+0x1c3) [0x7f136d3df383]\n[bt] (3) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*)+0x362) [0x7f136d2601e2]\n[

In [154]:
def build_model_twitter5_second(X_train, max_length=100,
                nb_filters=64,
                kernel_size=3,
                pool_size=2,
                regularization=0.01,
                weight_constraint=2.,
                dropout_prob=0.4,
                clear_session=True):
    if clear_session:
        K.clear_session()

    model = Sequential()
    
    model.add(Embedding(max_features+1, 50, input_length=X_train.shape[1]))
#     model.add(Embedding(
#         embeddings.shape[0],
#         embeddings.shape[1],
#         input_length=max_length,
#         trainable=False,
#         weights=[embeddings]))

    model.add(Conv1D(padding = "same", kernel_size = 3, filters = 32, activation = "relu"))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(LSTM(100))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model

In [155]:
new_model = build_model_twitter5_second(X_train)

In [156]:
X_train.shape, y.shape

((200000, 60), (2500000,))

In [159]:
new_model.fit(X_train, y, validation_split=0.1, nb_epoch=2, batch_size=128, verbose=1)

/usr/local/lib/python3.6/dist-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 180000 samples, validate on 20000 samples
Epoch 1/2
180000/180000 [==============================] - 464s - loss: 0.3730 - acc: 0.8248 - val_loss: 0.3372 - val_acc: 0.8501
Epoch 2/2
180000/180000 [==============================] - 462s - loss: 0.1581 - acc: 0.9374 - val_loss: 0.4390 - val_acc: 0.8236


In [163]:
test_6_labels

array([[0.9994729 ],
       [0.30984953],
       [0.7654236 ],
       ...,
       [0.99994326],
       [0.07081147],
       [0.99884915]], dtype=float32)

In [164]:
test_6_labels = new_model.predict_proba(X_test)

test_6_labels[test_6_labels >= 0.5] = 1
test_6_labels[test_6_labels < 0.5] = -1

test_6_labels_reversed = test_6_labels.copy()

test_6_labels_reversed[test_6_labels_reversed == -1] = 0
test_6_labels_reversed[test_6_labels_reversed == 1] = -1
test_6_labels_reversed[test_6_labels_reversed == 0] = 1

 9952/10000 [============================>.] - ETA: 0s

In [165]:
create_csv_submission(raw_data_test['id'], test_6_labels_reversed, 'NN_submission_2_partial_dataset.csv')